In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Flatten

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam 
import warnings
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 10,
                                   zoom_range = 0.25,
                                   horizontal_flip = True,
                                   vertical_flip=True,
                                   width_shift_range = 0.2,
                                   rotation_range=20,
                                   fill_mode = 'nearest')
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/ML_miniproject/archive (1)/chest_xray (1)/train (1)',
                                                 target_size = (224, 224),
                                                 color_mode="rgb",
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_datagen = ImageDataGenerator(rescale = 1/255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/ML_miniproject/archive (1)/chest_xray (1)/test (1)',
                                            target_size = (224, 224),
                                            color_mode="rgb",
                                            batch_size = 32,
                                            class_mode = 'binary')

val_datagen = ImageDataGenerator(rescale = 1/255)
val_set = test_datagen.flow_from_directory('/content/drive/MyDrive/ML_miniproject/archive (1)/chest_xray (1)/val (1)',
                                            target_size=(224,224),
                                           color_mode="rgb",
                                           batch_size = 2,
                                           class_mode = 'binary')

Found 5219 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=10)

In [ ]:
lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                          patience=8)

In [ ]:
def create_model2():
  model2 = Sequential()

  pretrained_model=tf.keras.applications.DenseNet201(
    include_top=False,
                    input_shape=(224,224,3),
                    pooling='avg',classes=2,
                    weights='imagenet')
  for layer in pretrained_model.layers:
          layer.trainable=False

  model2.add(pretrained_model)
  model2.add(Flatten())
  model2.add(Dense(512, activation='relu'))
  
  model2.add(Dense(1, activation='sigmoid'))
  model2.compile(optimizer=Adam(lr=0.001),loss='binary_crossentropy',metrics=['accuracy'])

  return model2

In [ ]:
densenet_model_new = create_model2()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
densenet_model_new.fit(x = training_set, validation_data=val_set,steps_per_epoch=100,callbacks=[early_stopping,lr], epochs = 5,batch_size=32)

Epoch 1/5
100/100 [==============================] - 771s 8s/step - loss: 0.2360 - accuracy: 0.9019 - val_loss: 0.8273 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 2/5
100/100 [==============================] - 760s 8s/step - loss: 0.1582 - accuracy: 0.9394 - val_loss: 0.5457 - val_accuracy: 0.8125 - lr: 0.0010
Epoch 3/5
100/100 [==============================] - 753s 8s/step - loss: 0.1498 - accuracy: 0.9413 - val_loss: 0.2429 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 4/5
100/100 [==============================] - 748s 7s/step - loss: 0.1435 - accuracy: 0.9464 - val_loss: 0.3158 - val_accuracy: 0.8125 - lr: 0.0010
Epoch 5/5
100/100 [==============================] - 735s 7s/step - loss: 0.1249 - accuracy: 0.9513 - val_loss: 0.1610 - val_accuracy: 0.9375 - lr: 0.0010


In [ ]:
from keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
mc

In [ ]:
densenet_model_new.save('final_model_densnet.h5') 

In [ ]:
import numpy as np
from keras.preprocessing import image
from google.colab import files
from keras.models import load_model
from io import BytesIO
from PIL import Image
model=load_model('final_model_densnet.h5')

In [ ]:
score = model.evaluate(training_set)

print("Train Loss: ", score[0])
print("Train Accuracy: ", score[1])

score = model.evaluate(test_set)
print("\nTest loss: ", score[0])
print("Test Accuracy: ", score[1])

164/164 [==============================] - 2828s 17s/step - loss: 0.1293 - accuracy: 0.9486
Train Loss:  0.12926560640335083
Train Accuracy:  0.9486491680145264
20/20 [==============================] - 319s 16s/step - loss: 0.2678 - accuracy: 0.8814

Test loss:  0.26779818534851074
Test Accuracy:  0.8814102411270142
